In [1]:
#!pip install torch torchvision

In [2]:
# 단순한 문자 RNN을 만들어보겠습니다.

import torch 
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [3]:
# 하이퍼파라미터 설정

n_hidden = 35 
lr = 0.01
epochs = 1000

In [4]:
# 사용하는 문자는 영어 소문자 및 몇가지 특수문자로 제한했습니다.
# alphabet(0-25), space(26), ... , start(0), end(1)

string = "hello pytorch. how long can a rnn cell remember? show me your limit!"
chars =  "abcdefghijklmnopqrstuvwxyz ?!.,:;01"

# 문자들을 리스트로 바꾸고 이의 길이(=문자의 개수)를 저장해놓습니다.
char_list = [i for i in chars]
n_letters = len(char_list)

In [5]:
# 문자를 그대로 쓰지않고 one-hot 벡터로 바꿔서 연산에 쓰도록 하겠습니다.

#Start = [0 0 0 … 1 0]
#a =     [1 0 0 … 0 0]
#b =     [0 1 0 … 0 0]
#c =     [0 0 1 … 0 0]
#...
#end =   [0 0 0 … 0 1]

In [6]:
# 문자열을 one-hot 벡터의 스택으로 만드는 함수
# abc -> [[1 0 0 … 0 0],
#         [0 1 0 … 0 0],
#         [0 0 1 … 0 0]]

def string_to_onehot(string):
    # 먼저 시작 토큰과 끝 토큰을 만들어줍니다.
    start = np.zeros(shape=n_letters ,dtype=int)
    end = np.zeros(shape=n_letters ,dtype=int)
    start[-2] = 1
    end[-1] = 1
    # 여기서부터는 문자열의 문자들을 차례대로 받아서 진행합니다.
    for i in string:
        # 먼저 문자가 몇번째 문자인지 찾습니다.
        # a:0, b:1, c:2,...
        idx = char_list.index(i)
        # 0으로만 구성된 배열을 만들어줍니다.
        # [0 0 0 … 0 0]
        zero = np.zeros(shape=n_letters ,dtype=int)
        # 해당 문자 인데스만 1로 바꿔줍니다.
        # b: [0 1 0 … 0 0]
        zero[idx]=1
        # start와 새로 생긴 zero를 붙이고 이를 start에 할당합니다.
        # 이게 반복되면 start에는 문자를 one-hot 벡터로 바꾼 배열들이 점점 쌓여가게 됩니다.
        start = np.vstack([start,zero])
    # 문자열이 다 끝나면 쌓아온 start와 end를 붙여줍니다.
    output = np.vstack([start,end])
    return output

In [7]:
# One-hot 벡터를 문자로 바꿔주는 함수 
# [1 0 0 ... 0 0] -> a 
# https://pytorch.org/docs/stable/tensors.html?highlight=numpy#torch.Tensor.numpy

def onehot_to_word(onehot_1):
    # 텐서를 입력으로 받아 넘파이 배열로 바꿔줍니다.
    onehot = torch.Tensor.numpy(onehot_1)
    # one-hot 벡터의 최대값(=1) 위치 인덱스로 문자를 찾습니다.
    return char_list[onehot.argmax()]

In [8]:
# RNN with 1 hidden layer

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.act_fn = nn.Tanh()
    
    def forward(self, input, hidden):
        # 입력과 hidden state를 cat함수로 붙여줍니다.
        combined = torch.cat((input, hidden), 1)
        # 붙인 값을 i2h 및 i2o에 통과시켜 hidden state는 업데이트, 결과값은 계산해줍니다.
        hidden = self.act_fn(self.i2h(combined))
        output = self.i2o(combined)
        return output, hidden
    
    # 아직 입력이 없을때(t=0)의 hidden state를 초기화해줍니다. 
    def init_hidden(self):
        return torch.zeros(1, self.hidden_size)
    
rnn = RNN(n_letters, n_hidden, n_letters)

In [9]:
# 손실함수와 최적화함수를 설정해줍니다.

loss_func = nn.MSELoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=lr)

In [10]:
# train

# 문자열을 onehot 벡터로 만들고 이를 토치 텐서로 바꿔줍니다.
# 또한 데이터타입도 학습에 맞게 바꿔줍니다.
one_hot = torch.from_numpy(string_to_onehot(string)).type_as(torch.FloatTensor())

for i in range(epochs):
    optimizer.zero_grad()
    # 학습에 앞서 hidden state를 초기화해줍니다.
    hidden = rnn.init_hidden()
    
    # 문자열 전체에 대한 손실을 구하기 위해 total_loss라는 변수를 만들어줍니다. 
    total_loss = 0
    for j in range(one_hot.size()[0]-1):
        # 입력은 앞에 글자 
        # pyotrch 에서 p y t o r c
        input_ = one_hot[j:j+1,:]
        # 목표값은 뒤에 글자
        # pytorch 에서 y t o r c h
        target = one_hot[j+1]
        output, hidden = rnn.forward(input_, hidden)
        
        loss = loss_func(output.view(-1),target.view(-1))
        total_loss += loss

    total_loss.backward()
    optimizer.step()

    if i % 100 == 0:
        print(total_loss)

tensor(2.7090, grad_fn=<AddBackward0>)
tensor(0.0740, grad_fn=<AddBackward0>)
tensor(0.0209, grad_fn=<AddBackward0>)
tensor(0.0124, grad_fn=<AddBackward0>)
tensor(0.0083, grad_fn=<AddBackward0>)
tensor(0.0077, grad_fn=<AddBackward0>)
tensor(0.0042, grad_fn=<AddBackward0>)
tensor(0.0034, grad_fn=<AddBackward0>)
tensor(0.0039, grad_fn=<AddBackward0>)
tensor(0.0028, grad_fn=<AddBackward0>)


In [11]:
# test 
# hidden state 는 처음 한번만 초기화해줍니다.

start = torch.zeros(1,n_letters)
start[:,-2] = 1

with torch.no_grad():
    hidden = rnn.init_hidden()
    # 처음 입력으로 start token을 전달해줍니다.
    input_ = start
    # output string에 문자들을 계속 붙여줍니다.
    output_string = ""

    # 원래는 end token이 나올때 까지 반복하는게 맞으나 끝나지 않아서 string의 길이로 정했습니다.
    for i in range(len(string)):
        output, hidden = rnn.forward(input_, hidden)
        # 결과값을 문자로 바꿔서 output_string에 붙여줍니다.
        output_string += onehot_to_word(output.data)
        # 또한 이번의 결과값이 다음의 입력값이 됩니다.
        input_ = output

print(output_string)

hello pytorch. how n cnonnn noyoyyyyyyyeeeeemmmmmrrrrrrrllllllllllll
